## Behandling av værdata

I denne filen behandler og manipulerer vi dataen slik at den blir lettere å lese og visualisere – den forberedes for videre analyse. 

In [1]:
import requests
from dotenv import load_dotenv
import os
import pandas as pd
import json

# Load environment variables from .env file
load_dotenv()

# Henter API-key/Client-id fra .env filen
client_id = os.getenv("client_id")

with open('../data/Byer.json', 'r') as f:
    byer = json.load(f)

byerAsString = ','.join(byer.values())

element_id = "mean(air_temperature P1D),sum(precipitation_amount P1D),mean(wind_speed P1D)"  # Temperatur, nedbør og vindhastighet
start_dato = "2022-01-01"  # Startdato
slutt_dato = "2023-01-01"  # Sluttdato

# Define endpoint and parameters
endpoint = 'https://frost.met.no/observations/v0.jsonld'
parameters = {
    "sources": byerAsString,
    "elements": element_id,
    "referencetime": f"{start_dato}/{slutt_dato}",
    "timeoffsets": "default",
    "timeresolutions": "P1D",
    "levels": "default"
}
print(f"Parametere: {parameters}")

# Sender forespørsel til Frost API
response = requests.get(endpoint, params=parameters, auth=(client_id, ''))

# Sjekker om forespørselen var vellykket
if response.status_code != 200:
    print(f"Error! Status code: {response.status_code}")
    print(f"Message: {response.json().get('error', {}).get('message', 'Ingen melding gitt')}")
    print(f"Reason: {response.json().get('error', {}).get('reason', 'Ingen årsak gitt')}")
    exit()

# Henter JSON-data
json_data = response.json()
if 'data' not in json_data:
    print("Ingen data funnet i forespørselen.")
    print(json_data)
    exit()

# Prosesserer data til en DataFrame
data = json_data['data']
rows = []  # Liste for å samle alle rader
for item in data:
    for observation in item['observations']:
        row = {
            'sourceId': item['sourceId'].split(':')[0],  # Fjern ':0' fra sourceId
            'referenceTime': item['referenceTime'],
            'elementId': observation['elementId'],
            'value': observation['value'],
            'unit': observation['unit'],
            'codequality': observation.get('qualityCode', '')  # Henter kvalitetskoden
        }
        rows.append(row)

# Opprett DataFrame fra rader
df = pd.DataFrame(rows)

# Legger til en ny kolonne for bynavn basert på sourceId
df['by'] = df['sourceId'].map({v: k for k, v in byer.items()})

# Forenkler tidsformatet (YYYY-MM-DD)
df['referenceTime'] = pd.to_datetime(df['referenceTime']).dt.date

# Splitter "elementId" i flere kolonner
df[['statistikk', 'variable']] = df['elementId'].str.extract(r'(\w+)\(([^)]+)')

# Gjør "codequality"-verdiene mer forståelige
quality_mapping = {0: "God", 1: "God", 2: "God", 
                   3: "Middels", 4: "Middels", 5: "Middels",
                   6: "Dårlig", 7: "Dårlig"}
df['codequality'] = df['codequality'].map(quality_mapping)

# Velger ønskede kolonner
df = df[['by', 'sourceId', 'referenceTime', 'statistikk', 'variable', 'value', 'unit', 'codequality']]

# Skriver DataFrame til en CSV-fil
data_fil = '../data/BehandletVaerData.csv'
df.to_csv(data_fil, index=False)
print(f"Data lagret i {data_fil}")

Parametere: {'sources': 'SN68230,SN18700,SN50540', 'elements': 'mean(air_temperature P1D),sum(precipitation_amount P1D),mean(wind_speed P1D)', 'referencetime': '2022-01-01/2023-01-01', 'timeoffsets': 'default', 'timeresolutions': 'P1D', 'levels': 'default'}
Data lagret i ../data/BehandletVaerData.csv


Her har vi gjort om kodekvaliteten fra tilfeldige tall til mer beskrivende tekst. I steden for at kodekvaliteten har et tall mellom 0 og 7, har vi endret det slik at kodekvalitet 0-2 skrives som "God", kodekvalitet 3-5 skrives som "Middels" og kodekvalitet 6 og 7 skrives som "Dårlig". 

In [2]:
import pandas as pd

# Last inn CSV-filen
data_fil = '../data/VaerData.csv'  # Tilpass denne stien til hvor CSV-filen faktisk er lagret
df = pd.read_csv(data_fil)

# Sjekk for manglende verdier
print("Antall manglende verdier per kolonne:")
print(df.isnull().sum())

# Hvis dere vil se hvilke rader som har manglende verdier:
print("\nRader med manglende verdier:")
print(df[df.isnull().any(axis=1)].index + 2)


Antall manglende verdier per kolonne:
by                0
sourceId          0
referenceTime     0
statistikk        0
variable          0
value             0
unit              0
codequality      10
dtype: int64

Rader med manglende verdier:
Index([287, 320, 1281, 1382, 1415, 1431, 1437, 1440, 1626, 2382], dtype='int64')


Her bruker vi Pandas til å identifisere manglende verdier i rådataen. Vi ser at det kun er i kodekvalitet-kolonnen at det er manglende verdier. Det er også listet opp hvilke rader som har manglende verdier. 

In [3]:
import pandas as pd

# Laster inn CSV-filen
data_fil = '../data/BehandletVaerData.csv' 
df = pd.read_csv(data_fil)

# Sjekker for manglende verdier
print("Manglende verdier per kolonne:")
print(df.isnull().sum())

# Fyller inn manglende 'codequality' med den vanligste verdien
most_frequent = df['codequality'].mode()[0]  # Finner vanligste verdi
df = df.assign(codequality=df['codequality'].fillna(most_frequent))
print(f"\nManglende 'codequality'-verdier fylt med: {most_frequent}")

# Sjekker på nytt for å bekrefte at det ikke er flere manglende verdier
print("\nEtter utfylling av manglende verdier:")
print(df.isnull().sum())

# Lagrer den oppdaterte DataFrame-en i CSV-filen
data_fil = '../data/BehandletVaerData.csv'
df.to_csv(data_fil, index=False)
print(f"\nOppdatert data lagret i {data_fil}")

# Rader med manglende verdier:
print("\nRader med manglende verdier:")
print(df[df.isnull().any(axis=1)].index + 2)

Manglende verdier per kolonne:
by                0
sourceId          0
referenceTime     0
statistikk        0
variable          0
value             0
unit              0
codequality      10
dtype: int64

Manglende 'codequality'-verdier fylt med: God

Etter utfylling av manglende verdier:
by               0
sourceId         0
referenceTime    0
statistikk       0
variable         0
value            0
unit             0
codequality      0
dtype: int64

Oppdatert data lagret i ../data/BehandletVaerData.csv

Rader med manglende verdier:
Index([], dtype='int64')


Etter vi analyserte og identifiserte hvilke rader som hadde manglende verdier, fylte vi inn verider der det opprinnelig manglet. Vi fant ut av hvilken kodekvalitet som var den hyppigste, og fylte inn den der det var manglende verdier. 